In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce

In [2]:
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [3]:

train = pd.read_csv("data/train1.csv")
test = pd.read_csv('data/test1.csv')
sub = pd.read_csv('data/sample.csv')

In [4]:
train.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales'],
      dtype='object')

In [5]:
import h2o
from h2o.automl import H2OAutoML

In [6]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.331-b09, mixed mode)
  Starting server from C:\Users\Wickkiey\anaconda3\envs\gpu\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Wickkiey\AppData\Local\Temp\tmpy0yj_jwq
  JVM stdout: C:\Users\Wickkiey\AppData\Local\Temp\tmpy0yj_jwq\h2o_Wickkiey_started_from_python.out
  JVM stderr: C:\Users\Wickkiey\AppData\Local\Temp\tmpy0yj_jwq\h2o_Wickkiey_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,29 days
H2O_cluster_name:,H2O_from_python_Wickkiey_kghf7x
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.531 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [18]:
train_h2o = h2o.H2OFrame(train)
test_h2o = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
y = 'Item_Outlet_Sales'
x = train_h2o.names
x.remove(y)

In [9]:
aml = H2OAutoML(max_models=7, seed=1)
aml.train(x=x, y=y, training_frame=train_h2o)

AutoML progress: |
14:54:00.139: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_1_AutoML_1_20220513_145400

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 564558.3565606193
RMSE: 751.3709846411554
MAE: 524.4007740834758
RMSLE: 0.4107127814240764
R^2: 0.8061135722494945
Mean Residual Deviance: 564558.3565606193
Null degrees of freedom: 8522
Residual degrees of freedom: 8518
Null deviance: 24817265080.348316
Residual deviance: 4811730872.966159
AIC: 137076.12540942448

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 1190569.8964397951
RMSE: 1091.1323918021108
MAE: 765.7814137612467
RMSLE: 0.5453948109041619
R^2: 0.5911222612764228
Mean Residual Deviance: 1190569.8964397951
Null degrees of freedom: 8522
Resid

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,7.657575e+02,1.702178e+01,7.617167e+02,7.453249e+02,7.909097e+02,7.722109e+02,7.586255e+02
1,mean_residual_deviance,1.190409e+06,4.838607e+04,1.176288e+06,1.126726e+06,1.259386e+06,1.207337e+06,1.182308e+06
2,mse,1.190409e+06,4.838607e+04,1.176288e+06,1.126726e+06,1.259386e+06,1.207337e+06,1.182308e+06
3,null_deviance,4.963925e+09,2.754540e+08,5.154757e+09,4.709137e+09,5.301692e+09,4.986066e+09,4.667971e+09
4,r2,5.911270e-01,7.715331e-03,5.956057e-01,5.913329e-01,5.906012e-01,5.788231e-01,5.992721e-01
5,residual_deviance,2.029492e+09,1.261262e+08,2.084381e+09,1.924448e+09,2.168662e+09,2.099559e+09,1.870411e+09
6,rmse,1.090879e+03,2.214764e+01,1.084568e+03,1.061474e+03,1.122224e+03,1.098789e+03,1.087340e+03
7,rmsle,5.457228e-01,9.457544e-03,5.388344e-01,5.345266e-01,5.588630e-01,5.479861e-01,5.484039e-01


In [10]:
pred = aml.leader.predict(test_h2o)

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [19]:
test_h2o['Item_Outlet_Sales'] = pred['predict']

In [20]:
sub.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1000
1,FDW14,OUT017,1000
2,NCN55,OUT010,1000
3,FDQ58,OUT017,1000
4,FDY38,OUT027,1000


In [23]:
h2o.export_file(test_h2o[['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']],'data/prediction.csv')

Export File progress: |██████████████████████████████████████████████████████████| (done) 100%
